In [1]:
import sys
sys.path.append('..')
from re import L
import torch
from torchvision import transforms, datasets, models
import clip
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import os
import kclip.model
import torch.optim as optim
import pytorch_model_summary
import numpy as np
tree_path = '../ImageNet21K/resources/imagenet21k_miil_tree.pth'
tree = torch.load(tree_path)
class_tree_list = tree['class_tree_list']
class_names = np.array(list(tree['class_description'].values()))
batch_size = 50

In [2]:
model, preprocess = clip.load("ViT-B/32")
image_path = '../dog_data/images/Images/'

In [3]:
dataset = datasets.ImageFolder(root=image_path, transform=preprocess)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True)

In [4]:
len(dataset)

20580

In [5]:
len(dataset.classes)

120

In [6]:
f = open("../ImageNet21K/animal_word.txt", 'r')
list_ = []
while True:
    line = f.readline().strip()
    if not line: break
    list_.append(line)

f.close()
# print(dict_.keys())
# print(len(dict_))
dog_text_list = []
for item in list_:
    if 'dog' in item:
        dog_text_list.append(item)
#         print(item)
# print(dog_text_list)
text_list = ['a photo of a '+ txt.replace("/", " of ") for txt in dog_text_list]
text_list = [txt.replace("_", " ") for txt in text_list]
len(text_list)

165

In [7]:
text_inputs = clip.tokenize(text_list).cuda()
text_features = model.encode_text(text_inputs)
text_features /= text_features.norm(dim=-1, keepdim=True)

In [8]:
for i,data in enumerate(dataloader,0):
    images , _ = data
    images = images.cuda()
    image_features = model.encode_image(images)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    if i == 0:
        break
print(image_features.shape)

torch.Size([50, 512])


In [9]:
print(text_features.shape)

torch.Size([165, 512])


In [10]:
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

In [11]:
similarity.shape

torch.Size([50, 165])

In [12]:
values, indices = similarity[0].topk(1)

In [20]:
label_dict = {}
for i in range(len(similarity)):
    values, indices = similarity[i].topk(20)
    for j in range(len(indices)):
        try:
            label_dict[indices[j].item()] += 1
        except:
            label_dict[indices[j].item()] = 1
sorted(label_dict.items(),  reverse=True, key = lambda item: item[1])


# print(text_list[10])
# print()
# for 

[(4, 29),
 (3, 28),
 (122, 23),
 (5, 19),
 (99, 18),
 (73, 17),
 (7, 17),
 (66, 15),
 (37, 13),
 (102, 12),
 (16, 12),
 (18, 12),
 (67, 12),
 (100, 11),
 (59, 11),
 (139, 11),
 (27, 11),
 (28, 10),
 (25, 10),
 (69, 10),
 (39, 10),
 (26, 10),
 (22, 9),
 (24, 9),
 (40, 9),
 (72, 9),
 (150, 9),
 (82, 9),
 (20, 9),
 (142, 9),
 (135, 8),
 (112, 8),
 (104, 8),
 (145, 8),
 (109, 8),
 (53, 8),
 (21, 8),
 (48, 8),
 (74, 8),
 (64, 8),
 (143, 8),
 (9, 8),
 (108, 8),
 (114, 7),
 (120, 7),
 (119, 7),
 (121, 7),
 (49, 7),
 (47, 7),
 (46, 7),
 (52, 7),
 (15, 7),
 (60, 7),
 (51, 7),
 (30, 7),
 (76, 7),
 (65, 7),
 (23, 7),
 (41, 7),
 (63, 7),
 (115, 7),
 (105, 7),
 (43, 7),
 (62, 7),
 (116, 6),
 (45, 6),
 (44, 6),
 (34, 6),
 (125, 6),
 (29, 6),
 (123, 6),
 (131, 6),
 (101, 6),
 (77, 6),
 (111, 6),
 (79, 6),
 (10, 6),
 (11, 6),
 (164, 6),
 (13, 6),
 (70, 6),
 (103, 6),
 (157, 6),
 (113, 5),
 (50, 5),
 (32, 5),
 (84, 5),
 (95, 5),
 (130, 5),
 (83, 5),
 (144, 5),
 (78, 5),
 (80, 5),
 (8, 5),
 (35, 5),
 (1

In [28]:
similarity.sum(dim=0).topk(50)

torch.return_types.topk(
values=tensor([1.5820, 1.4375, 1.3418, 1.2041, 1.1836, 1.1211, 1.0811, 0.9697, 0.9434,
        0.9028, 0.8892, 0.7251, 0.7241, 0.7119, 0.7051, 0.6880, 0.6553, 0.6528,
        0.6045, 0.5942, 0.5835, 0.5737, 0.5347, 0.5342, 0.5234, 0.5229, 0.5190,
        0.5083, 0.4902, 0.4839, 0.4670, 0.4651, 0.4626, 0.4531, 0.4531, 0.4504,
        0.4463, 0.4456, 0.3987, 0.3972, 0.3940, 0.3896, 0.3838, 0.3735, 0.3550,
        0.3408, 0.3384, 0.3364, 0.3335, 0.3259], device='cuda:0',
       dtype=torch.float16, grad_fn=<TopkBackward0>),
indices=tensor([160,  20, 143, 141,  21,  32, 120,  88, 139,   8,  86,  11,  70, 124,
         80, 121, 146,  15,  69,  24,  64, 113,  49, 115,  68,  17,  59,  27,
          4,   3,  67, 137,  44,  39, 128,   7, 112, 140,  50,  72,  66, 114,
        101,  47,  34, 131, 104, 144, 127,   5], device='cuda:0'))

In [45]:
for item in text_list:
    print(item)

a photo of a dogfish of shark of elasmobranch of fish of vertebrate of animal
a photo of a smoothhound of dogfish of shark of elasmobranch of fish of vertebrate of animal
a photo of a whitetip_shark of dogfish of shark of elasmobranch of fish of vertebrate of animal
a photo of a dog of domestic_animal of animal
a photo of a pooch of dog of domestic_animal of animal
a photo of a cur of dog of domestic_animal of animal
a photo of a lapdog of dog of domestic_animal of animal
a photo of a toy_dog of dog of domestic_animal of animal
a photo of a Chihuahua of toy_dog of dog of domestic_animal of animal
a photo of a Maltese_dog of toy_dog of dog of domestic_animal of animal
a photo of a Pekinese of toy_dog of dog of domestic_animal of animal
a photo of a Shih-Tzu of toy_dog of dog of domestic_animal of animal
a photo of a Blenheim_spaniel of toy_dog of dog of domestic_animal of animal
a photo of a King_Charles_spaniel of toy_dog of dog of domestic_animal of animal
a photo of a papillon of toy

In [51]:
text_list[16]

'a photo of a hunting_dog of dog of domestic_animal of animal'